# Example on how to add data to the dataframe (database)

Here we need to add new weather columns to the dataframe. Specifically:

```python
cols = [
    "precipitation",
    "wind_gusts_10m",
    "cloud_cover",
    'shortwave_radiation'
]
```

This data can be obtained from the same API so the data collection is strigtforward. 
First, we update `variables_standard` in Openmeteo class to add new quantities. This will assure that they are downloaded for all future data updates. However they also need to be added to the dataframe itself. This can be accomplised as follows. 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from glob import glob
from scipy.stats import fisk_gen

df_original = pd.read_parquet('../database/prev_latest.parquet')

In [ ]:
df_original.tail()

In [ ]:
start_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).first_valid_index())
today = pd.Timestamp(datetime.today())
end_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).last_valid_index())
from data_modules.collect_data_openmeteo import get_weather_data_from_api_forecast, get_weather_data_from_api, \
    locations, OpenMeteo

df_om_hist = get_weather_data_from_api(start_date, today-timedelta(hours=12), locations)



In [ ]:
df_om_forecast = get_weather_data_from_api_forecast(locations=locations)
if not df_om_forecast.columns.equals(df_om_hist.columns):
    print("! Error. Column mismatch between historical and forecasted weather!")
df_om = pd.concat([df_om_hist, df_om_forecast[df_om_hist.columns]], ignore_index=True)
df_om.drop_duplicates(subset='date', keep='last', inplace=True)
# df_om = process_weather_quantities(df_om, locations)
df_om.set_index('date',inplace=True)

In [ ]:
df_om

In [ ]:
# df_om.to_parquet('../database'+'db_openweather.parquet',engine='pyarrow')
    

In [ ]:
from data_modules.collect_data_openmeteo import OpenMeteo
for var in OpenMeteo.variables_standard:
    for col in df_original.columns.to_list():
        if str(col).__contains__(var):
            df_original.drop(col, axis=1, inplace=True)

In [ ]:
df_original

In [ ]:
df_original = df_original.join(df_om)

In [ ]:
df_original

# Add SMARD columns

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
# df_original = pd.read_parquet('../database/latest.parquet')

In [ ]:
df_original

In [ ]:
start_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).first_valid_index())
today = pd.Timestamp(datetime.today())
end_date = pd.Timestamp(df_original.dropna(how='any',inplace=False).last_valid_index())

In [ ]:
from data_modules.collect_data_smard import DataEnergySMARD
o_smard = DataEnergySMARD(start_date=start_date, end_date=end_date)
df_smard_flow = o_smard.get_international_flow()
df_smard_flow

In [ ]:
df_smard_flow.set_index('date',inplace=True)

In [ ]:

for col in df_smard_flow.columns.to_list():
    if not col in df_original.columns.to_list():
        print(f"Adding...{col}")
        # merge the new column with the dataframe 
        df_original = df_original.merge(df_smard_flow[[col]], how='left', left_index=True, right_index=True)
df_original.to_parquet('../database/latest.parquet')

In [ ]:
d

# Add new SMARD data (after aggregation change) and split to history and forecast

In [ ]:
df_hist = pd.read_parquet('../database/history.parquet')
last_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).last_valid_index())+timedelta(hours=1)
begin_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).first_valid_index())
print(last_ts, begin_ts)
df_hist

In [ ]:
from data_modules.collect_data_smard import DataEnergySMARD
o_smard = DataEnergySMARD(start_date=begin_ts, end_date=last_ts)
df_smard_flow = o_smard.get_international_flow()
df_smard_gen_forecasted = o_smard.get_forecasted_generation()
df_smard_con_forecasted = o_smard.get_forecasted_consumption()
df_smard = pd.merge(left=df_smard_flow,right=df_smard_gen_forecasted,left_on='date',right_on='date',how='outer')
df_smard = pd.merge(left=df_smard,right=df_smard_con_forecasted,left_on='date',right_on='date',how='outer')
df_smard.set_index('date',inplace=True)

In [ ]:
# drop not needed cols from df_hist
for col in df_hist.columns.to_list():
    if col in df_smard.columns.to_list() and not col in ['DA_auction_price']:
        df_hist.drop(col, axis=1, inplace=True)
# drop cols that are not there but are no longer needed
for col in df_hist.columns.to_list():
    if col.__contains__('_flow'):
        df_hist.drop(col, axis=1, inplace=True)
# add cols from new data to old data
for col in df_smard.columns.to_list():
    df_hist[col] = df_smard[col]
df_hist

In [ ]:
df_hist.to_parquet('../database/history.parquet')

# Add EPEXSPOT DATA FROM FILES TO DATAFRAME 

In [ ]:
import pandas as pd
from glob import glob
from datetime import datetime, timedelta

In [ ]:
df_hist = pd.read_parquet('../database/history.parquet')
last_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).last_valid_index())+timedelta(hours=1)
begin_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).first_valid_index())
print(last_ts, begin_ts)
df_hist

In [ ]:
raw_datadir = "../data/DE-LU/DayAhead_MRC/"
files = glob(raw_datadir + '*.csv')
df_da_upd = pd.DataFrame()
for file in files:
    df_i = pd.read_csv(file)
    df_da_upd = pd.concat([df_da_upd, df_i])
if len(files) == 0:
    raise FileNotFoundError(f"File in {raw_datadir} does not exist")
df_da_upd['date'] = pd.to_datetime(df_da_upd['date'])
df_da_upd.sort_values(by='date', inplace=True)
df_da_upd.drop_duplicates(subset='date', keep='first', inplace=True)
# for agreement with energy-charts
df_da_upd['date'] = df_da_upd['date'].dt.tz_localize('Etc/GMT-2').dt.tz_convert('UTC') #
df_da_upd.rename(columns={'Price':'DA_auction_price'},inplace=True)
# we do not need other data for now
df_da_upd = df_da_upd[['date','DA_auction_price']]
df_da_upd.set_index('date',inplace=True)

In [ ]:
df_hist = df_hist.fillna(df_da_upd)

In [ ]:
df_hist


In [ ]:
df_hist.to_parquet('../database/history.parquet')

# Split Data By Source 

In [1]:
import pandas as pd
from glob import glob
from datetime import datetime, timedelta

In [12]:
from data_modules.collect_data_openmeteo import get_weather_data_from_api_forecast, get_weather_data_from_api, \
    locations

In [22]:
df_hist = pd.read_parquet('../database/history.parquet')
last_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).last_valid_index())+timedelta(hours=1)
begin_ts = pd.Timestamp(df_hist.dropna(how='any',inplace=False).first_valid_index())
print(last_ts, begin_ts)
df_hist

2024-11-13 16:00:00+00:00 2015-10-01 00:00:00+00:00


,DA_auction_price,austria_export,austria_import,belgium_export,belgium_import,cloud_cover_fran,cloud_cover_hsee,cloud_cover_mun,cloud_cover_solw,cloud_cover_stut,...,wind_gusts_10m_mun,wind_gusts_10m_solw,wind_gusts_10m_stut,wind_offshore,wind_onshore,wind_speed_10m_fran,wind_speed_10m_hsee,wind_speed_10m_mun,wind_speed_10m_solw,wind_speed_10m_stut
date,,,,,,,,,,,,,,,,,,,,,
2015-10-01 00:00:00+00:00,35.83,4871.0,-571.0,0.0,0.0,26.699999,28.799999,0.0,0.0,0.6,...,34.200001,12.240000,30.960001,87.25,3873.75,13.104198,8.404285,15.328561,7.386582,14.458382
2015-10-01 01:00:00+00:00,29.14,4969.0,-704.0,0.0,0.0,10.799999,45.000000,0.0,0.0,0.0,...,31.680000,11.159999,29.519999,105.50,3561.00,13.392774,6.696387,14.512064,6.489992,14.512064
2015-10-01 02:00:00+00:00,29.40,4879.0,-655.0,0.0,0.0,1.800000,66.599998,0.0,0.9,0.0,...,30.239998,9.360000,29.160000,103.75,3417.00,13.104198,7.517021,14.408997,6.638072,15.137133
2015-10-01 03:00:00+00:00,32.77,5027.0,-478.0,0.0,0.0,0.000000,80.099998,0.0,6.3,0.0,...,31.680000,8.280000,30.239998,127.50,3413.75,13.479583,7.993298,15.281989,6.696387,15.778518
2015-10-01 04:00:00+00:00,44.97,5482.0,-905.0,0.0,0.0,0.000000,78.599998,0.6,5.1,0.0,...,32.760002,7.920000,30.960001,162.75,3415.25,13.684735,7.993298,16.243940,6.830519,15.463246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 11:00:00+00:00,162.07,0.0,0.0,0.0,0.0,100.000000,100.000000,100.0,100.0,98.0,...,12.959999,19.080000,15.480000,3555.50,3128.25,6.608722,31.765793,6.792466,11.165805,6.034700
2024-11-13 12:00:00+00:00,159.90,0.0,0.0,0.0,0.0,100.000000,97.000000,100.0,78.0,100.0,...,15.840000,15.840000,15.119999,3880.75,3514.50,6.489992,32.846916,6.193674,9.779817,5.991594
2024-11-13 13:00:00+00:00,165.60,0.0,0.0,0.0,0.0,100.000000,100.000000,100.0,65.0,100.0,...,14.040000,15.480000,13.320000,4030.25,3866.75,4.802999,36.445442,5.116561,8.311245,4.693825


In [9]:
df_epexspot = pd.DataFrame(df_hist['DA_auction_price'])
df_epexspot.to_parquet('../database/epexspot/history.parquet')

In [23]:
_df_ = get_weather_data_from_api_forecast(locations=locations)
df_om_hist = df_hist[_df_.set_index('date',inplace=False).columns]
df_om_hist.to_parquet('../database/openmeteo/history.parquet')

Processing forecast: EnBWHoheSee (windfarm) 0/5
Processing forecast: SolarparkWeesow-Willmersdorf  (solarfarm) 1/5
Processing forecast: Munchen (city) 2/5
Processing forecast: Stuttgart (city) 3/5
Processing forecast: Frankfurt (city) 4/5


In [31]:
df_smard_hist = df_hist[df_hist.columns.difference(df_om_hist.columns)]
df_smard_hist.drop('DA_auction_price', axis=1, inplace=True)
df_smard_hist.to_parquet('../database/smard/history.parquet')

/tmp/ipykernel_365030/305434584.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_smard_hist.drop('DA_auction_price', axis=1, inplace=True)


In [32]:
df_smard_hist

,austria_export,austria_import,belgium_export,belgium_import,czechia_export,czechia_import,denmark_export,denmark_import,france_export,france_import,...,residual_load_forecast,solar,sweden_export,sweden_import,switzerland_export,switzerland_import,total_gen,total_grid_load,wind_offshore,wind_onshore
date,,,,,,,,,,,,,,,,,,,,,
2015-10-01 00:00:00+00:00,4871.0,-571.0,0.0,0.0,563.0,-666.0,0.0,-935.0,3067.0,0.0,...,38197.50,0.00,0.0,0.0,1885.0,-9.0,56255.0,42158.50,87.25,3873.75
2015-10-01 01:00:00+00:00,4969.0,-704.0,0.0,0.0,574.0,-811.0,0.0,-935.0,2169.0,0.0,...,38809.75,0.00,0.0,0.0,1946.0,-19.0,55712.0,42476.25,105.50,3561.00
2015-10-01 02:00:00+00:00,4879.0,-655.0,0.0,0.0,556.0,-664.0,0.0,-935.0,1434.0,0.0,...,40323.25,0.00,0.0,0.0,1685.0,-15.0,56322.0,43844.00,103.75,3417.00
2015-10-01 03:00:00+00:00,5027.0,-478.0,0.0,0.0,548.0,-345.0,0.0,-935.0,1480.0,0.0,...,44235.25,0.00,0.0,0.0,1885.0,-10.0,61640.0,47776.50,127.50,3413.75
2015-10-01 04:00:00+00:00,5482.0,-905.0,0.0,0.0,581.0,-256.0,0.0,-935.0,1629.0,-124.0,...,52732.50,2.75,0.0,0.0,1717.0,-83.0,66348.0,56313.25,162.75,3415.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 11:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54979.25,4482.25,0.0,0.0,0.0,0.0,55610.0,66145.25,3555.50,3128.25
2024-11-13 12:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,53958.75,3637.00,0.0,0.0,0.0,0.0,55280.0,64991.00,3880.75,3514.50
2024-11-13 13:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,53423.50,2255.00,0.0,0.0,0.0,0.0,53724.0,63575.50,4030.25,3866.75


In [18]:
df_epexspot = pd.read_parquet('../database/openmeteo/history.parquet')
df_epexspot

,temperature_2m_hsee,relative_humidity_2m_hsee,surface_pressure_hsee,wind_speed_10m_hsee,wind_direction_10m_hsee,precipitation_hsee,wind_gusts_10m_hsee,cloud_cover_hsee,shortwave_radiation_hsee,temperature_2m_solw,...,shortwave_radiation_stut,temperature_2m_fran,relative_humidity_2m_fran,surface_pressure_fran,wind_speed_10m_fran,wind_direction_10m_fran,precipitation_fran,wind_gusts_10m_fran,cloud_cover_fran,shortwave_radiation_fran
date,,,,,,,,,,,,,,,,,,,,,
2015-10-01 00:00:00+00:00,14.00,75.818497,1036.000000,8.404285,46.735668,0.0,16.559999,28.799999,0.00,6.6630,...,0.00,9.254,64.508926,1017.952087,13.104198,52.815220,0.0,24.480000,26.699999,0.00
2015-10-01 01:00:00+00:00,13.90,76.311859,1035.699951,6.696387,36.253937,0.0,12.599999,45.000000,0.00,6.2630,...,0.00,8.754,66.957550,1017.435974,13.392774,53.746078,0.0,24.480000,10.799999,0.00
2015-10-01 02:00:00+00:00,13.80,77.066528,1035.500000,7.517021,16.699326,0.0,11.159999,66.599998,0.00,6.0130,...,0.00,8.404,69.048912,1017.222839,13.104198,52.815220,0.0,24.840000,1.800000,0.00
2015-10-01 03:00:00+00:00,13.65,77.561058,1034.900024,7.993298,7.765082,0.0,11.520000,80.099998,0.00,5.7130,...,0.00,8.104,70.718048,1017.110291,13.479583,55.885483,0.0,25.919998,0.000000,0.00
2015-10-01 04:00:00+00:00,13.55,77.545570,1034.599976,7.993298,352.234924,0.0,12.240000,78.599998,0.00,5.4630,...,0.00,7.854,71.676331,1016.802856,13.684735,54.637459,0.0,27.000000,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13 11:00:00+00:00,12.30,84.000000,1031.000000,31.765793,321.441559,0.0,37.439999,100.000000,68.25,6.5915,...,129.00,7.143,84.000000,1018.745728,6.608722,29.357658,0.0,16.199999,100.000000,103.00
2024-11-13 12:00:00+00:00,12.15,80.000000,1030.599976,32.846916,313.667847,0.0,42.119999,97.000000,74.75,7.1415,...,140.25,7.543,81.000000,1018.368896,6.489992,19.440128,0.0,15.480000,100.000000,91.75
2024-11-13 13:00:00+00:00,12.05,83.000000,1030.300049,36.445442,317.802521,0.0,45.000000,100.000000,93.75,7.3415,...,87.75,7.593,81.000000,1017.877319,4.802999,12.994630,0.2,15.119999,100.000000,63.75


In [15]:
df_om_hist = df_hist[_df_.set_index('date',inplace=True).columns]
df_om_hist.to_parquet('../database/openmeteo/history.parquet')

AttributeError: 'NoneType' object has no attribute 'columns'